In [2]:
import warnings
warnings.filterwarnings('ignore')

import torch
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt

# Load pre-trained model tokenizer (vocabulary)
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [9]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert

In [10]:
from collections import defaultdict, Counter
from sklearn.model_selection import KFold
from sklearn.svm import SVC, LinearSVC, NuSVC
from collections import defaultdict
from nltk.corpus import stopwords

import os
import glob
import codecs
import scispacy
import spacy
import sklearn
import warnings 

In [11]:
import numpy as np
import pandas as pd

data = pd.read_csv("mtsamples_t2.csv")

temp = [i for i in data.index if pd.isna(data["transcription"][i])]
data = data.drop(temp)
data = data.reset_index(drop=True)
print(data["medical_specialty"].unique())

d = {'group_1': 1, 'group_2': 2, 'group_3': 3}
label = data['medical_specialty'].map(d, na_action='ignore')
print(label)

['group_3' 'group_2' 'group_1']
0       3
1       2
2       2
3       1
4       1
       ..
4961    3
4962    3
4963    3
4964    3
4965    3
Name: medical_specialty, Length: 4966, dtype: int64


In [12]:
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)


# Loading tokenizer from the bert layer
PRE_TRAINED_MODEL_NAME = 'emilyalsentzer/Bio_ClinicalBERT'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [32]:
import string  
punct = set(string.punctuation)
nlp = spacy.load("en_core_sci_sm")
med7 = spacy.load("en_core_med7_lg")

In [55]:
def tokenize_doc(text):
    bow = defaultdict(float)
    temp = []
    doc = nlp(text)
    doc = doc.ents
    lowered_tokens = map(lambda t: str(t).lower(), doc)

    for token in lowered_tokens:
        temp.append(token)
#     temp = getNGrams(temp,5)
    temp = " ".join(temp)
    return temp

In [56]:
tokeniz_s = [tokenize_doc(x) for x in data['transcription']]
# data['transcription'].apply((lambda x: tokenize_doc(x) ))

In [57]:
print(tokeniz_s[0])

subjective white female complaint allergies allergies lived seattle thinks claritin zyrtec short time lose effectiveness allegra summer weeks working prescription nasal sprays asthma doest daily medication flaring up.,medications medication tri-cyclen medicine allergies.,objective:,vitals weight pounds blood pressure 124/78.,heent throat erythematous exudate nasal mucosa erythematous swollen drainage tms clear.,neck supple adenopathy.,lungs clear.,assessment allergic rhinitis.,plan:,1 zyrtec allegra again loratadine prescription coverage cheaper.,2 samples nasonex nostril weeks prescription


In [58]:
# tokenized = pd.Series(tokenized)
# tokenized = tokenized.apply((lambda x: tokenizer.convert_tokens_to_ids(tokenizer.tokenize(x))))
tokenized = [tokenizer.convert_tokens_to_ids(tokenizer.tokenize(x))[0:512] for x in tokeniz_s]


# Reshaping the array:
max_len = 0
for i in tokenized:
    if len(i) > max_len:
        max_len = len(i)
print(max_len)

512


In [62]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [69]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tokeniz_s, label, test_size=0.33, random_state=42)

In [70]:
print(X_train[0])

preoperative diagnoses:,1 right shoulder rotator glenohumeral rotator arthroscopy.,3 degenerative joint diagnoses:,1 right shoulder rotator glenohumeral rotator arthroscopy.,3 degenerative joint performed right shoulder hemiarthroplasty.,anesthesia general.,estimated blood loss cc.,complications none.,components depuy global shoulder system stem cemented depuy articulating head used.,brief history patient right-hand dominant female shoulder pain years affecting daily living function pain conservative treatment.,procedure patient operative suite operative field department anesthesia administered anesthetic patient beach chair position care padded right upper extremity prepped draped usual sterile fashion deltopectoral approach skin blade blunt dissection mayo scissors overlying subscapular tendon bursal tissue perforating bleeders bovie hemostasis bursa rongeur subscapular tendon rotator subacromial region evaluated rotator irreparable bone greater tuberosity articular surface biceps te

In [71]:
MAX_LEN = 510

In [72]:
train_input = bert_encode(X_train, tokenizer, max_len= MAX_LEN)
# encode  test set 
test_input = bert_encode(X_test, tokenizer, max_len= MAX_LEN )
train_labels = y_train

In [73]:
from tensorflow.keras.layers import  Input
input_word_ids = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_word_ids")
input_mask = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_mask")
segment_ids = Input(shape=(MAX_LEN,), dtype=tf.int32, name="segment_ids")

In [74]:
from tensorflow.keras.layers import Dense
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])  
clf_output = sequence_output[:, 0, :]
out = Dense(1, activation='sigmoid')(clf_output)  

In [75]:
# intilize model
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
model.compile(Adam(lr=2e-6), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 510)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 510)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 510)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]      

In [76]:
# train
train_history = model.fit(
    train_input, train_labels,
    validation_split=0.2
    # epochs=2
)


84/84 [==============================] - 4185s 50s/step - loss: 4.0338 - accuracy: 0.0000e+00 - val_loss: 4.0630 - val_accuracy: 0.0000e+00


In [77]:
test_pred = model.predict(test_input)
preds = test_pred.round().astype(int)
preds

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])